In [5]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [6]:
data = pd.read_csv("County_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv")
# data = pd.read_csv("Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv")
data

,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,2000-01-31,...,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31
0,3101,0,Los Angeles County,county,CA,CA,"Los Angeles-Long Beach-Anaheim, CA",6,37,205554.642143,...,842828.125468,843458.023150,846590.488371,848657.557658,852267.676628,857411.008912,863957.219674,869036.446028,873116.490139,876489.981962
1,139,1,Cook County,county,IL,IL,"Chicago-Naperville-Elgin, IL-IN-WI",17,31,146088.131319,...,295891.891394,298514.510384,300331.667498,301377.039259,302029.073728,302846.384103,303511.080209,303864.098187,304254.182907,304914.611846
2,1090,2,Harris County,county,TX,TX,"Houston-The Woodlands-Sugar Land, TX",48,201,109502.880748,...,282099.985083,283142.592334,283733.994057,283615.859587,283246.105196,282907.172194,282692.273148,282325.752609,281874.816774,281642.532453
3,2402,3,Maricopa County,county,AZ,AZ,"Phoenix-Mesa-Chandler, AZ",4,13,143224.384045,...,467120.843290,468892.771578,470292.903957,470461.256487,469857.613304,468743.383602,467798.713506,467070.117625,466405.864875,465828.629769
4,2841,4,San Diego County,county,CA,CA,"San Diego-Chula Vista-Carlsbad, CA",6,73,213034.659393,...,911324.080512,921505.174834,930221.439635,934686.727793,936123.029271,935687.520148,935242.345948,934533.913892,934725.657369,935232.664342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3068,846,3206,Banner County,county,NE,NE,"Scottsbluff, NE",31,7,NaN,...,321125.066597,324004.896959,327386.466115,328069.339347,330165.658998,336736.175154,346631.949352,354998.496627,358646.215997,363180.460914
3069,1648,3207,Daggett County,county,UT,UT,NaN,49,9,NaN,...,321388.066928,321448.379443,320133.501640,318038.022608,315829.862113,315766.403474,318169.837605,322748.075604,326448.913102,330270.364833
3070,1432,3208,Thomas County,county,NE,NE,NaN,31,171,NaN,...,162545.502097,163043.652064,163376.474022,163708.115429,163511.719807,164047.103362,164787.407684,166054.747665,166257.891113,166435.634476
3071,2794,3212,McPherson County,county,NE,NE,"North Platte, NE",31,117,NaN,...,296002.925100,301582.530690,306218.863367,308687.099937,311291.959869,318725.125071,329326.702388,339183.091984,344650.536347,350922.241689


In [7]:
dates = data.columns[9:] # These are the timestamps
price_by_county = pd.DataFrame(data[dates].values.T, columns=data['RegionName'], index=dates) # Transposing the dataset, now using the RegionName (ZIP code) as the columns, dates for the index, and transposing the housing data
# price_by_county.columns = dates['RegionName']
# price_by_county
price_by_county.to_csv('zip_code_housing_labels.csv')
# labels = data[dates]
# labels.index = labels['RegionName']
# labels.drop(['RegionName'], axis=1)
# labels.to_csv('housing_labels.csv')

In [8]:
price_by_county

RegionName,Los Angeles County,Cook County,Harris County,Maricopa County,San Diego County,Orange County,Kings County,Miami-Dade County,Dallas County,Riverside County,...,Wheeler County,Hooker County,San Juan County,Treasure County,Loup County,Banner County,Daggett County,Thomas County,McPherson County,Petroleum County
2000-01-31,205554.642143,146088.131319,109502.880748,143224.384045,213034.659393,2.483530e+05,200639.071495,119668.621108,93990.540943,149547.902766,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-02-29,205787.076303,146058.811433,109474.767351,143516.508250,213878.120425,2.505451e+05,201602.627381,120083.060160,94042.350458,149886.943085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-03-31,206645.170206,146299.372635,109313.922526,143902.521074,214900.510511,2.521021e+05,202654.141877,120436.828134,94119.405750,150300.974680,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-04-30,208344.164074,146991.131284,109247.781175,144696.080612,217215.437980,2.552860e+05,204936.099271,121153.716601,94297.411981,151248.156464,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-05-31,210504.871387,147876.373157,109221.904554,145570.897200,219715.973772,2.580317e+05,207366.081909,121787.881479,94530.243952,152486.072483,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31,857411.008912,302846.384103,282907.172194,468743.383602,935687.520148,1.123102e+06,846570.558934,531887.523252,312967.856389,604212.620117,...,230463.934800,123386.090629,528411.317244,185303.801165,310436.823833,336736.175154,315766.403474,164047.103362,318725.125071,158653.532384
2024-09-30,863957.219674,303511.080209,282692.273148,467798.713506,935242.345948,1.130097e+06,849704.713103,533291.737261,312401.969738,605260.053955,...,233933.291898,123872.022476,530029.921353,185987.546582,315129.516712,346631.949352,318169.837605,164787.407684,329326.702388,161280.339942
2024-10-31,869036.446028,303864.098187,282325.752609,467070.117625,934533.913892,1.136611e+06,851903.458103,533542.030514,311843.578865,605860.776709,...,238367.449855,124136.559601,532923.729681,187909.810346,320006.941594,354998.496627,322748.075604,166054.747665,339183.091984,165289.537724
2024-11-30,873116.490139,304254.182907,281874.816774,466405.864875,934725.657369,1.143075e+06,854880.300260,532797.604892,311106.260896,606543.758767,...,241060.345103,123836.154642,531769.821932,188521.945064,321877.019649,358646.215997,326448.913102,166257.891113,344650.536347,168560.091132
